In [1]:
import os
from agno.storage.agent.sqlite import SqliteAgentStorage
from agno.memory.db.sqlite import SqliteMemoryDb
from agno.agent import Agent
from agno.tools.sql import SQLTools
from agno.models.ollama import Ollama
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()


* Models

In [2]:
ollama_model=Ollama(id="llama3.2:3b")


* database tool

In [3]:
db_url = 'sqlite:///../app/db/shop.db'
engine = create_engine(db_url)

In [5]:
tool = SQLTools(db_engine=engine)
tool

<SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>

In [6]:
tool.functions

OrderedDict([('list_tables',
              Function(name='list_tables', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=None, entrypoint=<bound method SQLTools.list_tables of <SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>>, sanitize_arguments=True, show_result=False, stop_after_tool_call=False, pre_hook=None, post_hook=None)),
             ('describe_table',
              Function(name='describe_table', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=None, entrypoint=<bound method SQLTools.describe_table of <SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>>, sanitize_arguments=True, show_result=False, stop_after_tool_call=False, pre_hook=None, post_hook=None)),
             ('run_sql_query',
              Function(name='run_sql_query', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=Non

In [7]:
tool.list_tables()

'["Customers", "OrderDetails", "Orders", "Products"]'

In [8]:
tool.describe_table(table_name="Products")

'["{\'name\': \'ProductID\', \'type\': INTEGER(), \'nullable\': False, \'default\': None, \'primary_key\': 1}", "{\'name\': \'Name\', \'type\': VARCHAR(), \'nullable\': False, \'default\': None, \'primary_key\': 0}", "{\'name\': \'Description\', \'type\': VARCHAR(), \'nullable\': True, \'default\': None, \'primary_key\': 0}", "{\'name\': \'Price\', \'type\': FLOAT(), \'nullable\': False, \'default\': None, \'primary_key\': 0}", "{\'name\': \'StockQuantity\', \'type\': INTEGER(), \'nullable\': False, \'default\': None, \'primary_key\': 0}", "{\'name\': \'Category\', \'type\': VARCHAR(), \'nullable\': False, \'default\': None, \'primary_key\': 0}"]'

In [9]:
tool.run_sql(sql="SELECT * FROM Customers", limit=1)

[{'CustomerID': 1,
  'FirstName': 'Juan',
  'LastName': 'Pérez',
  'Email': 'juan.perez@email.com',
  'Phone': '555-1234',
  'Address': 'Calle de la Rosa 12, Sevilla'}]

In [10]:
tool.run_sql(sql="SELECT * FROM Products", limit=1)

[{'ProductID': 1,
  'Name': 'Perfume Chanel No. 5',
  'Description': 'Perfume floral y aldehídico, uno de los más emblemáticos',
  'Price': 120.99,
  'StockQuantity': 50,
  'Category': 'Women'}]

* SQL Agent

In [12]:

sql_agent = Agent(model=ollama_model, 
              description="You are an expert in SQL. You can create sql queries from natural language.",
              name="sql_agent", 
              tools=[tool.functions],
              show_tool_calls=True,
              instructions="""You are a SQL expert focused on writing precise, efficient queries.
                  When a user messages you, determine if you need query the database or can respond directly.
                  If you can respond directly, do so."""
        )

response = sql_agent.run("Select the first row from the Products table")
print(response.content)

SELECT * FROM Products LIMIT 1;


In [13]:
response.messages

[Message(role='system', content='You are an expert in SQL. You can create sql queries from natural language.\n\n<instructions>\nYou are a SQL expert focused on writing precise, efficient queries.\n                  When a user messages you, determine if you need query the database or can respond directly.\n                  If you can respond directly, do so.\n</instructions>', name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, audio_output=None, reasoning_content=None, tool_name=None, tool_args=None, tool_call_error=None, stop_after_tool_call=False, add_to_agent_memory=True, from_history=False, metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, prompt_tokens=0, completion_tokens=0, prompt_tokens_details=None, completion_tokens_details=None, additional_metrics=None, time=None, time_to_first_token=None, timer=None), references=None, created_at=1739736820),
 Message(role='user', content='Select the first row from the Products table',

In [14]:
response.metrics

{'input_tokens': [97],
 'output_tokens': [9],
 'total_tokens': [106],
 'prompt_tokens': [0],
 'completion_tokens': [0],
 'additional_metrics': [{'total_duration': 15595496200,
   'load_duration': 4107458800,
   'prompt_eval_duration': 9518000000,
   'eval_duration': 1247000000}],
 'time': [15.899510699993698]}

In [16]:
response  = sql_agent.run("Number of rows in the Customers table")
print(response.content)

The number of rows in the `Customers` table is:

```sql
SELECT COUNT(*) FROM Customers;
```

Result: (answer depends on actual data)


In [17]:
response.messages

[Message(role='system', content='You are an expert in SQL. You can create sql queries from natural language.\n\n<instructions>\nYou are a SQL expert focused on writing precise, efficient queries.\n                  When a user messages you, determine if you need query the database or can respond directly.\n                  If you can respond directly, do so.\n</instructions>', name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, audio_output=None, reasoning_content=None, tool_name=None, tool_args=None, tool_call_error=None, stop_after_tool_call=False, add_to_agent_memory=True, from_history=False, metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, prompt_tokens=0, completion_tokens=0, prompt_tokens_details=None, completion_tokens_details=None, additional_metrics=None, time=None, time_to_first_token=None, timer=None), references=None, created_at=1739736902),
 Message(role='user', content='Number of rows in the Customers table', name=N

Observacion: No esta llamando a las tools

### SQL Agent

In [18]:
import json
semantic_model = {
    "tables": [
        {
            "table_name": "Customers",
            "table_description": "Contains information about the customers, including their contact details, address, and unique customer identifier.",
            "Use Case": "Use this table to store and retrieve customer details, which is essential for order processing, tracking, and communication.",
        },
        {
            "table_name": "Products",
            "table_description": "Contains information about the products available for sale, including their names, descriptions, prices, and stock levels.",
            "Use Case": "Use this table to manage and retrieve data about products, including product details and availability, for inventory or product analysis.",
        },
        {
            "table_name": "Orders",
            "table_description": "Records customer orders, linking them to specific customers, their order dates, and the total amount of the order.",
            "Use Case": "Use this table to track customer orders, their dates, and total amounts. This table helps to analyze sales performance and order history.",
        },
        {
            "table_name": "OrderDetails",
            "table_description": "Stores information about individual items in each order, including the products ordered, quantities, and the subtotal for each item.",
            "Use Case": "Use this table to get details of items within each order, allowing for deeper analysis of product sales, order composition, and inventory management.",
        },
    ]
}

semantic_model_str = json.dumps(semantic_model, indent=2)
semantic_model_str

'{\n  "tables": [\n    {\n      "table_name": "Customers",\n      "table_description": "Contains information about the customers, including their contact details, address, and unique customer identifier.",\n      "Use Case": "Use this table to store and retrieve customer details, which is essential for order processing, tracking, and communication."\n    },\n    {\n      "table_name": "Products",\n      "table_description": "Contains information about the products available for sale, including their names, descriptions, prices, and stock levels.",\n      "Use Case": "Use this table to manage and retrieve data about products, including product details and availability, for inventory or product analysis."\n    },\n    {\n      "table_name": "Orders",\n      "table_description": "Records customer orders, linking them to specific customers, their order dates, and the total amount of the order.",\n      "Use Case": "Use this table to track customer orders, their dates, and total amounts. Th

In [24]:
#model=Ollama(id="qwen2.5-coder:7b",structured_outputs=True)

tools = SQLTools(db_url=db_url)
sql_agent = Agent(model=ollama_model, 
              description="You are an expert in SQL and you can use tools to create sql queries from natural language.",
              name="sql_agent", 
              tools=[SQLTools(db_engine=engine, db_url=db_url, tables=semantic_model).functions],
              show_tool_calls=True,
              instructions="""You are a SQL expert focused on writing precise, efficient queries.
                When a user messages you, determine if you need query the database or can respond directly.
                If you can respond directly, do so.
                1. First identify the tables you need to query from `list_tables` tool
                2. Then use `describe_table` tool to get the details of the selected table
                3. Then, using all the information available, create one single syntactically correct SQL query to accomplish your task
                If you need to join tables, check the `semantic_model` for the relationships between the tables.
                4. Then use `run_sql_query` tool to get the results
                5.After you run the query, analyse the results and return the answer in markdown format.
                """,
                additional_context=f"""
                The following `semantic_model` contains information about tables and the relationships between them.
                <semantic_model>
                {semantic_model_str}
                </semantic_model
                -- Customers table
                CREATE TABLE IF NOT EXISTS Customers(
                        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
                        FirstName TEXT NOT NULL,
                        LastName TEXT NOT NULL,
                        Email TEXT UNIQUE NOT NULL,
                        Phone TEXT,
                        Address TEXT
                );

                --  Products table
                CREATE TABLE IF NOT EXISTS Products (
                        ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
                        Name TEXT NOT NULL,
                        Description TEXT,
                        Price REAL NOT NULL,
                        StockQuantity INTEGER NOT NULL,
                        Category TEXT NOT NULL
                );


                --  Orders table
                CREATE TABLE IF NOT EXISTS Orders (
                        OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
                        CustomerID INTEGER NOT NULL,
                        OrderDate DATE NOT NULL,
                        TotalAmount REAL NOT NULL,
                        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
                );

                --  OrderDetails table
                CREATE TABLE IF NOT EXISTS OrderDetails (
                        OrderDetailID INTEGER PRIMARY KEY AUTOINCREMENT,
                        OrderID INTEGER NOT NULL,
                        ProductID INTEGER NOT NULL,
                        Quantity INTEGER NOT NULL,
                        Subtotal REAL NOT NULL,
                        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
                        FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
                );
        """
        )


In [25]:
response = sql_agent.run("SELECT * FROM Products LIMIT 1")
print(response.content)

**Direct Response**

You are looking for the first product in our database. Since we don't have any data, I'll provide you with some sample data.

Here is the first product:

| ProductID | Name        | Description          | Price | StockQuantity | Category |
|-----------|-------------|----------------------|-------|---------------|----------|
| 1         | iPhone 13   | Latest iPhone model   | 999.99| 10           | Electronics |

Let me know if you have any further questions or if there's anything else I can help you with!


In [27]:
response.messages

[Message(role='system', content='You are an expert in SQL and you can use tools to create sql queries from natural language.\n\n<instructions>\nYou are a SQL expert focused on writing precise, efficient queries.\n                When a user messages you, determine if you need query the database or can respond directly.\n                If you can respond directly, do so.\n                1. First identify the tables you need to query from `list_tables` tool\n                2. Then use `describe_table` tool to get the details of the selected table\n                3. Then, using all the information available, create one single syntactically correct SQL query to accomplish your task\n                If you need to join tables, check the `semantic_model` for the relationships between the tables.\n                4. Then use `run_sql_query` tool to get the results\n                5.After you run the query, analyse the results and return the answer in markdown format.\n                \n<